<a href="https://colab.research.google.com/github/knc6/jarvis-tools-notebooks/blob/master/jarvis-tools-notebooks/DiffractGPT_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[DiffractGPT: Atomic Structure Determination from X‑ray DiffractionPatterns Using a Generative Pretrained Transformer](https://pubs.acs.org/doi/epdf/10.1021/acs.jpclett.4c03137)

GitHub: https://github.com/atomgptlab/atomgpt

YouTube Demo: https://www.youtube.com/watch?v=tyz4MVh2_7k&list=PLjf6vHVv7AoInTVQmfNSMs_12DBXYCcDd&index=2

Author email: kchoudh2@jhu.edu




Following notebook shows examples how to get atmoic structure from powder X-dar diffraction from generative model DiffractGPT and Structure matching methods

In [ ]:
# !pip install -q condacolab
# import condacolab
# condacolab.install()

In [1]:
%%time
import os
os.chdir('/content')
!rm -rf Software
os.makedirs('/content/Software')
os.chdir('/content/Software')
!pip install uv
if not os.path.exists('atomgpt'):
  !rm -rf atomgpt
  !git clone https://github.com/atomgptlab/atomgpt.git
  os.chdir('atomgpt')
  !pip install -q -e .



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 122.6 MB/s eta 0:00:00
Cloning into 'atomgpt'...
remote: Enumerating objects: 1529, done.
remote: Counting objects: 100% (115/115), done.
remote: Compressing objects: 100% (64/64), done.
remote: Total 1529 (delta 68), reused 58 (delta 50), pack-reused 1414 (from 2)
Receiving objects: 100% (1529/1529), 67.26 MiB | 24.65 MiB/s, done.
Resolving deltas: 100% (868/868), done.
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.5/104.5 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 329.0 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.2/67.2 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [2]:
# Check imports
import atomgpt
import os
os.environ.pop('MPLBACKEND', None)  # Remove the invalid backend
import matplotlib
matplotlib.use('Agg')  # Use a compatible backend
import matplotlib.pyplot as plt



In [3]:
!pip install -q triton==3.2.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.2/253.2 MB 5.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.7.0 requires triton==3.3.0; platform_system == "Linux" and platform_machine == "x86_64", but you have triton 3.2.0 which is incompatible.
torchaudio 2.8.0+cu126 requires torch==2.8.0, but you have torch 2.7.0 which is incompatible.
atomgpt 2025.10.7 requires triton==3.3.0, but you have triton 3.2.0 which is incompatible.


# Inference Example for LaB6


| Argument        | Description |
|----------------|-------------|
| `--model_name` | HuggingFace-compatible model ID for the pretrained DiffractGPT model. Example: `knc6/diffractgpt_mistral_chemical_formula`. |
| `--formula`    | The chemical formula for which the XRD pattern will be generated. Example: `LaB6`. |
| `--dat_path`   | Output path for the generated `.dat` file. The file contains two columns: 2θ (angle) and intensity. |
| `--config_path`| Path to a JSON config file containing generation parameters (use the example file). |


Theoretical XRD data atomgpt/examples/inverse_model_multi/my_data.dat

In [4]:
os.chdir('/content/Software/atomgpt')

Theoretical XRD data example. Header is chemical formula, the 2theta and intensity values.

In [2]:
file_content="""LaB6
21.3849 7.2634
30.4238 11.5522
37.4904 4.8924
43.5638 2.4932
49.0228 5.697
54.0619 3.0706
63.3067 1.1376
67.6436 3.2224
71.8493 2.2997
75.9558 1.451
83.9744 0.9538
87.9297 2.0058
99.8122 1.7793
103.8433 1.5299
107.9457 0.6619
112.1457 0.9822
116.4753 2.1496
120.9747 1.1266
130.7195 0.8452
136.154 1.0326
142.1905 4.7341
149.1934 2.7378
"""
with open("my_file.dat","w") as f:
  f.write(file_content)

In [7]:
%%time
!python atomgpt/inverse_models/inverse_predict.py --model_name knc6/diffractgpt_mistral_chemical_formula --formula LaB6 --dat_path my_file.dat --config_path atomgpt/examples/inverse_model_multi/config.json

2025-10-24 12:50:33.823202: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1761310233.849488    3694 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1761310233.858001    3694 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1761310233.888897    3694 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1761310233.888925    3694 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1761310233.888931    3694 computation_placer.cc:177] computation placer alr

## Let's visualize the structure

In [9]:
!pip install "numpy<2"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 89.4 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.2.6
    Uninstalling numpy-2.2.6:
      Successfully uninstalled numpy-2.2.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 2.2.3 which is incompatible.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.3 which is incompatible.
opencv-python-headless 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.
jax 0.7.2 requires numpy>=2.0, but you have numpy 1.26.4 which is incompatible.
gradio 5.49.1 requires h

In [1]:
from jarvis.io.vasp.inputs import Poscar
from ase.visualize import view

pos="""System
1.0
4.16 0.0 0.0
0.0 4.16 0.0
0.0 0.0 4.16
La B
1 6
direct
0.0 0.0 0.0 La
0.5 0.5 0.0 B
0.5 0.0 0.5 B
0.0 0.5 0.5 B
0.5 0.0 0.0 B
0.0 0.5 0.0 B
0.0 0.0 0.5 B
"""
atoms = Poscar.from_string(pos).atoms
view(atoms.make_supercell([2,2,2]).ase_converter(),viewer='x3d')

## Structure matching based XRD option using AtomGPT.org API (AGAPI)

In [3]:
!pip install agapi


In [4]:
from agapi.client import Agapi
# Go to Website: https://atomgpt.org/
# Navigate to Profile >> Settings >> Account >> API Keys >> Show/Create
# It will look like sk-xxxxxxxxxxxxx paste below
api_key='sk-xxxxxxx'
client = Agapi(api_key=api_key)
r = client.pxrd_query(file_path="my_file.dat")
print(r)

System
1.0
4.154998579020728 0.0 0.0
0.0 4.154998579020728 -0.0
0.0 0.0 4.154998579020728
La B 
1 6 
direct
0.0 0.0 0.0 La
0.8002152990241852 0.5 0.5 B
0.1997847009758148 0.5 0.5 B
0.5 0.5 0.8002152990241852 B
0.5 0.5 0.1997847009758148 B
0.5 0.1997847009758148 0.5 B
0.5 0.8002152990241852 0.5 B

